In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../dataset/PlantVillage/pepper_bell",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE

)

Found 2475 files belonging to 2 classes.


In [4]:
pepper_bell_classes=dataset.class_names
print(pepper_bell_classes)
len(pepper_bell_classes)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']


2

In [5]:
data_size_len=len(dataset)
data_size_len

78

In [6]:
train_size=int(.8*data_size_len)
test_size=int(.1*data_size_len)
validation_size=data_size_len-train_size-test_size
print(train_size+test_size+validation_size)
print(train_size,test_size,validation_size)

78
62 7 9


In [7]:
train_data=dataset.take(train_size)
test_data=dataset.skip(train_size).take(test_size)
validation_data=dataset.skip(train_size+test_size)
print(len(train_data),len(validation_data))

62 9


In [8]:
resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  tf.keras.layers.Rescaling(1./255),
])

In [9]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
])

In [10]:
train_ds = train_data.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

### Model 1 : try without augmentation 

In [11]:
INPUT_SHAPE=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes=len(pepper_bell_classes)
model = tf.keras.Sequential([
    resize_and_rescale,
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=INPUT_SHAPE)

c:\Users\baps\OneDrive\Desktop\Projects\aiml\practice\aimlenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [12]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (32, 254, 254, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 516128)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 64)               │    33,032,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 2)                │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,033,282 (126.01 MB)

 Trainable params: 33,033,282 (126.01 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [14]:
history = model.fit(
    train_data,
    batch_size=BATCH_SIZE,
    validation_data=validation_data,
    verbose=1,
    epochs=4,
)

Epoch 1/4
62/62 ━━━━━━━━━━━━━━━━━━━━ 50s 790ms/step - accuracy: 0.5412 - loss: 8.8543 - val_accuracy: 0.5880 - val_loss: 0.6894
Epoch 2/4
62/62 ━━━━━━━━━━━━━━━━━━━━ 45s 729ms/step - accuracy: 0.6104 - loss: 0.6874 - val_accuracy: 0.6142 - val_loss: 0.6844
Epoch 3/4
62/62 ━━━━━━━━━━━━━━━━━━━━ 45s 726ms/step - accuracy: 0.6113 - loss: 0.6832 - val_accuracy: 0.5918 - val_loss: 0.6834
Epoch 4/4
62/62 ━━━━━━━━━━━━━━━━━━━━ 45s 731ms/step - accuracy: 0.6124 - loss: 0.6793 - val_accuracy: 0.6217 - val_loss: 0.6749


In [15]:
model.evaluate(test_data)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.5875 - loss: 0.6813


[0.6834431290626526, 0.5758928656578064]

#### testwith augmented test data

In [16]:
model.evaluate(test_data.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)
)

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 309ms/step - accuracy: 0.5721 - loss: 0.6841


[0.6850758194923401, 0.5669642686843872]

### Model 2 :try with augmantation

In [17]:
train_augmented_data=train_data.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

test_augmented_data=test_data.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)


validation_augmented_data=validation_data.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)


In [18]:
INPUT_SHAPE=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes=len(pepper_bell_classes)
model_with_augmentation = tf.keras.Sequential([
    resize_and_rescale,
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax'),
])
model_with_augmentation.build(input_shape=INPUT_SHAPE)

In [19]:
model_with_augmentation.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [20]:
history = model_with_augmentation.fit(
    train_augmented_data,
    batch_size=BATCH_SIZE,
    validation_data=validation_augmented_data,
    verbose=1,
    epochs=10,
)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 56s 891ms/step - accuracy: 0.6215 - loss: 6.0973 - val_accuracy: 0.8352 - val_loss: 0.3606
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 56s 895ms/step - accuracy: 0.8777 - loss: 0.2859 - val_accuracy: 0.8876 - val_loss: 0.2476
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 886ms/step - accuracy: 0.8974 - loss: 0.2624 - val_accuracy: 0.8914 - val_loss: 0.2773
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 892ms/step - accuracy: 0.9260 - loss: 0.2042 - val_accuracy: 0.9401 - val_loss: 0.2073
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 881ms/step - accuracy: 0.9372 - loss: 0.1762 - val_accuracy: 0.9476 - val_loss: 0.1817
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 888ms/step - accuracy: 0.9374 - loss: 0.1809 - val_accuracy: 0.9476 - val_loss: 0.1806
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 880ms/step - accuracy: 0.9381 - loss: 0.1614 - val_accuracy: 0.9625 - val_loss: 0.1501
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 55s 877ms/step - accuracy: 0.9432 - loss: 0.1603 - val_accu

In [21]:
scores = model.evaluate(test_augmented_data)
scores

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 314ms/step - accuracy: 0.6016 - loss: 0.6787


[0.6793612837791443, 0.5982142686843872]

### model 3 : add some layers in model

In [22]:
model3_with_augmentation = tf.keras.Sequential([
    resize_and_rescale,
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation='relu', input_shape=INPUT_SHAPE),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax'),
])
model3_with_augmentation.build(input_shape=INPUT_SHAPE)

In [23]:
model3_with_augmentation.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
history = model3_with_augmentation.fit(
    train_augmented_data,
    batch_size=BATCH_SIZE,
    validation_data=validation_augmented_data,
    verbose=1,
    epochs=20,
)

Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 59s 932ms/step - accuracy: 0.6727 - loss: 0.7316 - val_accuracy: 0.8876 - val_loss: 0.3170
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 59s 943ms/step - accuracy: 0.9133 - loss: 0.2314 - val_accuracy: 0.9401 - val_loss: 0.1755
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 60s 956ms/step - accuracy: 0.9432 - loss: 0.1742 - val_accuracy: 0.9363 - val_loss: 0.1866
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 59s 954ms/step - accuracy: 0.9551 - loss: 0.1401 - val_accuracy: 0.9700 - val_loss: 0.0850
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 62s 992ms/step - accuracy: 0.9640 - loss: 0.1462 - val_accuracy: 0.9401 - val_loss: 0.2162
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 58s 924ms/step - accuracy: 0.9540 - loss: 0.1767 - val_accuracy: 0.9663 - val_loss: 0.0859
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 57s 915ms/step - accuracy: 0.9766 - loss: 0.0794 - val_accuracy: 0.9363 - val_loss: 0.1559
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 57s 918ms/step - accuracy: 0.9737 - loss: 0.0720 - val_accu

In [24]:
scores = model3_with_augmentation.evaluate(test_augmented_data)
scores

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 335ms/step - accuracy: 1.0000 - loss: 0.0031


[0.004098949953913689, 1.0]

In [25]:
model3_with_augmentation.save("../models/pepperBell.h5")